In [1]:

import torch
import torch.utils.data
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger

import chess
from network import ChessBoardEvalNN
from data.dataset import ChessPositionsDataset
from data.database import Database

import data.database

%load_ext autoreload
# %load_ext line_profiler
%autoreload 2

In [ ]:
with Database() as db:
    db.store_positions_from_pgn_file("./data/pgn/comp-2019-03.pgn", max_games = 300_000)

In [ ]:


with Database() as db:
    count = db.cur.execute("SELECT COUNT(*) FROM games").fetchone()[0]
    print(f"{count} games stored")
    count = db.cur.execute("SELECT COUNT(*) FROM positions").fetchone()[0]
    print(f"{count} positions stored")


In [2]:
dataset = ChessPositionsDataset(num_positions = 8_414_441)
# print(dataset[0])
val_set, train_set = torch.utils.data.random_split(dataset, [414_441, 8_000_000], generator=torch.Generator().manual_seed(42))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.displot(dataset.get_all_evaluations())

In [7]:
import multiprocessing


hparams = {"learning_rate": 1e-3}

model : ChessBoardEvalNN = ChessBoardEvalNN(hparams=hparams)

# model = ChessBoardEvalNN.load_model(hparams)

logger = TensorBoardLogger("./tb_logs", name="ChessNN", default_hp_metric=False, log_graph=True)

trainer = pl.Trainer(
    max_epochs=20,
    accelerator="gpu" if torch.cuda.is_available() else None,
    logger = logger,
    log_every_n_steps=1
)

train_dataloader = DataLoader(train_set, batch_size=1024, num_workers=multiprocessing.cpu_count(), shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=1024, num_workers=multiprocessing.cpu_count())

trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params | In sizes      | Out sizes
----------------------------------------------------------------------------
0 | intake        | Sequential    | 0      | [1, 12, 8, 8] | [1, 768] 
1 | residual_1    | ResidualBlock | 1.2 M  | [1, 768]      | [1, 768] 
2 | residual_2    | ResidualBlock | 1.2 M  | [1, 768]      | [1, 768] 
3 | residual_3    | ResidualBlock | 1.2 M  | [1, 768]      | [1, 768] 
4 | residual_4    | ResidualBlock | 1.2 M  | [1, 768]      | [1, 768] 
5 | exhaust_score | Sequential    | 429 K  | [1, 768]      | [1, 1]   
----------------------------------------------------------------------------
5.2 M     Trainable params
0         Non-trainable params
5.2 M     Total params
20.665    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Exception in thread Thread-92:
Traceback (most recent call last):
  File "/home/volk/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/volk/.cache/pypoetry/virtualenvs/chessai-9CBJxCkL-py3.9/lib/python3.9/site-packages/tensorboard/summary/writer/event_file_writer.py", line 233, in run
    self._record_writer.write(data)
  File "/home/volk/.cache/pypoetry/virtualenvs/chessai-9CBJxCkL-py3.9/lib/python3.9/site-packages/tensorboard/summary/writer/record_writer.py", line 40, in write
    self._writer.write(header + header_crc + data + footer_crc)
  File "/home/volk/.cache/pypoetry/virtualenvs/chessai-9CBJxCkL-py3.9/lib/python3.9/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 766, in write
    self.fs.append(self.filename, file_content, self.binary_mode)
  File "/home/volk/.cache/pypoetry/virtualenvs/chessai-9CBJxCkL-py3.9/lib/python3.9/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 160, in append
    s

In [5]:

for i in range(2000, 2020):
    board, target_id, evaluation = dataset[i]
    board = board.to(model.device)
    board = board[None, :]
    model.eval()
    moves, score = model.forward(board)
    print(f"truth eval: {evaluation}")
    print(f"predicted eval: {score}")
    print(f"truth move: {target_id.argmax()}")
    print(f"predicted move: {torch.topk(moves.flatten(), 5).indices} | prob: {moves[torch.topk(moves.flatten(), 5).indices]}")

model.save_model("models/model1.model")

truth eval: 0.617796003818512
predicted eval: 0.4998486042022705
truth move: 1380
predicted move: tensor([4095,  261, 1487, 2325, 1357], device='cuda:0') | prob: tensor([9.8092e-01, 1.8391e-02, 5.0878e-05, 4.6642e-05, 2.9457e-05],
       device='cuda:0', grad_fn=<IndexBackward0>)
truth eval: 0.6103960275650024
predicted eval: 0.49834194779396057
truth move: 1611
predicted move: tensor([4095,  197, 4094, 3518,  966], device='cuda:0') | prob: tensor([9.9997e-01, 5.4104e-08, 3.9486e-08, 2.9684e-08, 2.4361e-08],
       device='cuda:0', grad_fn=<IndexBackward0>)
truth eval: 0.6749959588050842
predicted eval: 0.49888426065444946
truth move: 3837
predicted move: tensor([4095,  197, 4094, 3518, 1869], device='cuda:0') | prob: tensor([9.9994e-01, 2.0211e-07, 1.4285e-07, 1.1251e-07, 9.8393e-08],
       device='cuda:0', grad_fn=<IndexBackward0>)
truth eval: 0.666595995426178
predicted eval: 0.4934867322444916
truth move: 131
predicted move: tensor([4095,  197, 4094, 3518,  838], device='cuda:0') 